# Yquoter: Unified Financial Data Interface - Basic Usage
## Introduction
Welcome to the basic usage guide for Yquoter, a comprehensive toolkit designed to unify and analyze financial data from CN/HK/US markets, leveraging multiple sources including custom spiders and TuShare.

This notebook demonstrates the core functionality of the public API.
## 1. Environment Setup and Configuration
This section covers necessary setup steps, including importing the library, initializing external dependencies (like TuShare), and configuring internal settings (like data sources and caching).
**Imports**

In [ ]:
import pandas as pd
import warnings
import os
from src.yquoter import (
    init_tushare,
    init_cache_manager,
    register_source,
    register_tushare_module,
    set_default_source,
    get_stock_profile,
    get_stock_history,
    get_ma_n,
    get_boll_n,
    get_max_drawdown,
    get_vol_ratio,
    get_rsi_n,
    get_stock_realtime,
    get_stock_financials,
    get_stock_factors,
    get_newest_df_path,
    get_rv_n
)

# Configuration for display
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', category=FutureWarning)

print("Yquoter modules imported successfully.")

**Configure External Services and Cache**

In [ ]:
# Set up TuShare API token. This is essential if you rely on TuShare for data.
# Replace 'YOUR_TUSHARE_TOKEN' with your actual token.
TUSHARE_TOKEN = 'YOUR_TUSHARE_TOKEN'
init_tushare(token=TUSHARE_TOKEN)
print(f"TuShare initialized.")


# Register a custom data source or module (e.g., if you have a custom spider)
# Since the actual implementation of the custom source is not available, we use placeholders.
# register_source(source_name='my_spider', func_type='realtime')

# If you wanna activate tuShare Module, just use the function below.
# register_tushare_module() 

# Set the maximum number of entries to keep in the cache to manage memory/disk space.
MAX_CACHE_ENTRIES = 100
init_cache_manager(MAX_CACHE_ENTRIES)
print(f"Cache limit set to {MAX_CACHE_ENTRIES} entries.")

## 2. Core Data Acquisition
We will use `Kweichow Moutai (600519)` as the example stock for the Chinese A-share market.

### 2.1. Stock Profile (Static Information)
**Retrieve static profile information (code, name, industry, main business, etc.).**

In [ ]:
STOCK_CODE = "600519"
MARKET = "CN"

print(f"--- Fetching Profile for {STOCK_CODE} ---")

# get_stock_profile(code, market, source)
profile_df = get_stock_profile(code=STOCK_CODE, market=MARKET)

# Display main business with truncation for better notebook formatting
if not profile_df.empty and 'main_business' in profile_df.columns:
    profile_df['main_business_short'] = profile_df['main_business'].str[:80] + '...'
    display_cols = ['code', 'name', 'industry', 'listing_date', 'main_business_short']
    display(profile_df[display_cols])
else:
    display(profile_df)

### 2.2. Historical and Realtime Data (OHLCV)
**Retrieve daily historical data and the latest trade information.**

In [ ]:
# get_stock_history(market, code, start, end, klt, fqt, fields, source)
# klt could be in str (such as "d", "M") or in int (from 101 to 105)
START_DATE = '20230101'
END_DATE = '20240930'

print(f"--- Fetching Historical Data ({START_DATE} to {END_DATE}) ---")
history_df = get_stock_history(
    market=MARKET, 
    code=STOCK_CODE, 
    start=START_DATE, 
    end=END_DATE,
    klt="d"
)  # You can also add a param, fields, which can be "basic" or "full". 

print(f"Total {len(history_df)} historical records retrieved.")
display(history_df.head(5))


print(f"\n--- Fetching Realtime Data for {STOCK_CODE} ---")
# You can also get Realtime data of different code (which must be in the same market) by passing a list of codes.
# get_stock_realtime(market, codes, fields, source) 
realtime_df = get_stock_realtime(market=MARKET, codes=STOCK_CODE)
display(realtime_df)

### 2.3. Financial Statements and Fundamental Factors
**Retrieve periodic financial statements (e.g., quarterly) and calculated fundamental factors.**

In [ ]:
# get_stock_financials(market, code, end_day, report_type, limit, source)
# Example: Quarterly Income Statement (LRB)
print(f"--- Fetching Latest 4 Quarterly Income Statements (LRB) ---")
financials_df = get_stock_financials(
    market=MARKET, 
    code=STOCK_CODE, 
    end_day=END_DATE,
    report_type='LRB', 
    limit=4
)
display(financials_df[['REPORT_DATE', 'TOTAL_OPERATE_INCOME', 'PARENT_NETPROFIT']])


# get_stock_factors(market, code, trade_date, source)
# This version only can get some certain factors, and more options will be supported in the next version.
print(f"\n--- Fetching Factors ---")
factors_df = get_stock_factors(
    code=STOCK_CODE, 
    market=MARKET, 
    trade_date='20240930'
)
display(factors_df.tail(5))

## 3. Technical Indicators and Analysis
**Using the historical data fetched in Section 2.2, we demonstrate how to use the built-in technical analysis functions.**

In [ ]:
if 'close' in history_df.columns:
    
    # 3.1. Moving Average (MA)
    # You can use the below function without a param because it will auto-matching the latest local cache.
    # Only passing a df is also allowed.
    # get_ma_n (market=None, code=None, start=None, end=None, n=5, df=None) 
    print("--- Calculating 20-Day Moving Average (MA20) ---")
    ma_df = get_ma_n(n=20, history_df.copy())
    display(ma_df[['date', 'MA20']].tail(5))


    # 3.2. Bollinger Bands (BOLL)
    # get_boll_n (market=None, code=None, start=None, end=None, n=20, df=None)
    print("\n--- Calculating 20-Day Bollinger Bands ---")
    boll_df = get_boll_n(n=20, history_df.copy())
    display(boll_df[['date', 'upper', 'mid', 'lower']].tail(5))
    
    
    # 3.3. Relative Strength Index (RSI)
    # get_rsi_n(market=None, code=None,start=None, end=None, n=5, df=None)
    print("\n--- Calculating 14-Day Relative Strength Index (RSI14) ---")
    rsi_df = get_rsi_n(n=14, history_df.copy())
    display(rsi_df[['date', 'RSI14']].tail(5))

    # 3.4. Rolling Volatility (RV)
    # get_rv_n(market=None, code=None, start=None, end=None, n=5, df=None)
    print("\n--- Calculating 5-Day Rolling Volatility (RV5) ---")
    rv_df = get_rv_n(n=5, history_df.copy())
    # Assuming the output column is standardized as 'RV_5' for display consistency
    display(rv_df[['date', 'rv5']].tail(5))

else:
    print("Historical data not available or missing 'close' column for analysis.")

## 4. Utility and Risk Metrics
**Demonstrate utility functions for risk assessment and file management.**

In [ ]:
# 4.1. Max Drawdown
# get_max_drawdown(market=None, code=None, start=None, end=None, n=5, df=None)
print("--- Calculating Maximum Drawdown (Based on Close Price) ---")
if 'close' in history_df.columns:
    max_dd = get_max_drawdown(history_df)
    print(f"Maximum Drawdown occurred: {max_dd['max_drawdown_peak_date'].iloc[0]} to {max_dd['max_drawdown_trough_date'].iloc[0]}")
    print(f"Max Drawdown Percentage: {max_dd['max_drawdown'].iloc[0]:.2%}")
else:
    print("Cannot calculate Max Drawdown: 'close' column is missing.")


# 4.2. Volume Ratio
# get_vol_ratio(market=None, code=None, start=None, end=None, n=20, df=None)
print("\n--- Calculating 10-Day Volume Ratio (VS previous 10 days) ---")
if 'vol' in history_df.columns:
    vol_ratio_df = get_vol_ratio(n=10, history_df.copy())
    display(vol_ratio_df[['date', 'vol', 'vol_ratio10']].tail(5))
else:
    print("Cannot calculate Volume Ratio: 'vol' column is missing.")


# 4.3. Cache and File Path Utility
# get_newest_df_path()
print("\n--- Retrieving Newest Cache File Path (Utility) ---")
# Example: Get the path of the latest cache file.
try:
    cache_path = get_newest_df_path()
    print(f"Cache path for historical data: {cache_path}")
    if os.path.exists(cache_path):
        print("File exists locally.")
    else:
        print("File does not exist locally yet (or cache is disabled).")
except Exception as e:
    print(f"Error retrieving cache path: {e}")